## Mining the 360 Giving database for information related to Bereavement Services: building the dataset

This notebook sets out the steps for building the dataset used in the analysis on grant funding for bereavement related services. The database used is the 360Giving database called GrantNav https://grantnav.threesixtygiving.org/

### Step 1: Extracting the data from GrantNav
The following search terms were agreed: bereaved, bereavement, bereave*, grief, grieving, widowed. No wildcard search is possible in GrantNav. "Bereaved" and "bereavement" gave the same results, based on the root "bereave". So four data extracts were downloaded in CSV files to manipulate further.

### Step 2: Read data extracts into Jupyter notebook, concatonate and de-duplicate
Read in the data; check shapes; concatonate; de-duplicate to arrive at a single dataframe.

### Step 3: Drop early data; focus on past 5 years.

We have looked at the last five financial years, not including the current financial year. (2015/6 to 2019/20).


In [1]:
# Import the modules and packages 

from platform import python_version # be clear about version of python
import pandas as pd # pandas is a package for data wrangling, manipulation & analysis
import matplotlib.pyplot as plt #plotting package
import datetime as dt

print("1. Succesfully imported necessary modules")    
print("")

1. Succesfully imported necessary modules



In [2]:
print("python version==%s" % python_version())
print("pandas==%s" % pd.__version__)


python version==3.6.5
pandas==0.23.0


In [3]:
#Presenting non trunctated output
#Set presentation options
pd.options.display.max_columns = 80 # to avoid truncated output 
pd.options.display.max_rows = 250 #
pd.options.display.max_colwidth = 250 # avoid truncating cell content

### Step 2: Read data extracts into Jupyter Notebook, merge and de-duplicate

In [4]:
#Set up a list of the files containting the data extracts of interest
GrantNav_datafiles = [
    "grantnav_bereaved.csv",
    "grantnav_grief.csv",
    "grantnav_grieving.csv",
    "grantnav_widowed.csv"
    ]
#Create empty dictionary
data = {}
# Iterate over the list, drop file ending, add to dictionary
for f in GrantNav_datafiles:
    #tell it where to go look for the data
    d = pd.read_csv("C:/Users/sonja/Desktop/Life_Admin/Bereavement_Services/GrantNav_data/{0}".format(f))
    key_name = f.replace(".csv", "")
    data[key_name] = d
    

In [5]:
#Let's look at the shape of these four data extracts
for k in data:
    print(data[k].shape) 

(1733, 79)
(185, 79)
(38, 79)
(2052, 79)


We have 79 columns in each data extract; this is as expected.  
We are going to concatonate the four (expecting 4008 rows) before deduping

In [6]:
#Loop through each key in the data dictionary. 
#Display first five rows of the dataframe associated with the key.
#for k in data:
 #   print(data[k].head(3)) 

In [7]:
concat_data = pd.concat([data ["grantnav_bereaved"], data ["grantnav_grief"], 
                    data ["grantnav_grieving"], data ["grantnav_widowed"]], sort = False)

In [8]:
concat_data.shape

(4008, 79)

In [9]:
#De-Dupe the dataframe

data_deduped = concat_data.drop_duplicates() # default is to use all of the columns to identify duplicates which 
#fine for our purposes here

In [10]:
data_deduped.shape # we have lost a few hundred rows

(3863, 79)

In [11]:
# Replace the space in all the column headers
data_deduped.columns = data_deduped.columns.str.replace(' ', '_')

### Step 3: Focus on last 5 years 

In [12]:
#Need to ensure the award date column is a datetime object (rather than some other type of data type)
data_deduped ["Award_Date"] = pd.to_datetime( data_deduped ["Award_Date"])
# This error message doesn't seem to have affected the output - the col content is now datetime rather than 
#and object. So we carry on.

C:\Users\sonja\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [13]:
data_deduped.dtypes
#Checked it's a datetime data type

Identifier                                                                                                                                                                                                                                                                                                      object
Title                                                                                                                                                                                                                                                                                                           object
Description                                                                                                                                                                                                                                                                                                     object
Currency                                                           

In [14]:
# Select rows where the date is between 31 March 2015 and 1 April 2020

data_5yrs = data_deduped[data_deduped.Award_Date.between('2015-03-31', '2020-04-01')]

In [15]:
data_5yrs.shape

(1139, 79)

In [16]:
#Export to csv
data_5yrs.to_csv ('C:/Users/sonja/Desktop/Life_Admin/Bereavement_Services/data_5yrs.csv', 
                  index = False, header=True)